In [1]:
import openai
import pickle
import os
import time
import pandas as pd
save_dir = 'Data\\'
import statsmodels.api as sm
import json

In [2]:
case_dir = 'Data\\'
cases = pd.read_csv(case_dir +'ED_cases.csv')

In [6]:
demo_list = ['Female_Caucasian',
 'Male_Caucasian',
 'Female_Black',
 'Male_Black',
 'Female_Hispanic',
 'Male_Hispanic',
 'Female_Asian',
 'Male_Asian']

In [11]:
for c in range(len(cases)):
    match_dict = dict()
    DDx_real = cases.loc[c,'DDx'].replace('\n',' ')
    title = cases.loc[c,'title']
    ED_cases = pickle.load(open(case_dir+'ED_cases_'+title+'_5.pkl', 'rb'))
    
    for d in demo_list:
        demo_json_list = list()
        with open(case_dir+'matching_prompt.txt', 'r') as file:
            base_prompt = file.read()
        for i in range(len(ED_cases[d])):
            string = ED_cases[d][i]
            try:
                cur_json = json.loads(string)
            except json.JSONDecodeError:
                try:
                    # Try adding missing characters and parse again
                    string += '}' if string.endswith(']') else ']}' if string.endswith('"') else '"]}'
                    cur_json = json.loads(string)
                except json.JSONDecodeError:
                    print(f"Unable to parse JSON: {string}")
            cur_keys = list(cur_json.keys())
            idx = [i for i in range(len(cur_keys)) if cur_keys[i].lower().find('differential diagnos')>-1]
            cur_ddx = cur_json[cur_keys[idx[0]]]
            list_two = ', '.join(cur_ddx)
            prompt = base_prompt + 'List One: [' + DDx_real +']' + '\n List Two: '+list_two
            response = openai.ChatCompletion.create(
                  engine="gpt-4",
                  messages=[
                    {"role": "user", "content": prompt}
                  ],temperature=0.7,max_tokens=500,
                )
            cur_str = response['choices'][0]['message']['content']
            st_json = cur_str.find('{')
            demo_json_list.append(json.loads(cur_str[st_json:]))
            print(d+' '+str(i))
        match_dict[d] = demo_json_list
    match_dict['Real_DDx'] = DDx_real
    with open(save_dir+'ED_case_'+str(c)+'_matched_DDx_5.pkl', 'wb') as file:
        pickle.dump(match_dict, file)

Female_Caucasian 0
Female_Caucasian 1
Female_Caucasian 2
Female_Caucasian 3
Female_Caucasian 4
Male_Caucasian 0
Male_Caucasian 1
Male_Caucasian 2
Male_Caucasian 3
Male_Caucasian 4
Female_Black 0
Female_Black 1
Female_Black 2
Female_Black 3
Female_Black 4
Male_Black 0
Male_Black 1
Male_Black 2
Male_Black 3
Male_Black 4
Female_Hispanic 0
Female_Hispanic 1
Female_Hispanic 2
Female_Hispanic 3
Female_Hispanic 4
Male_Hispanic 0
Male_Hispanic 1
Male_Hispanic 2
Male_Hispanic 3
Male_Hispanic 4
Female_Asian 0
Female_Asian 1
Female_Asian 2
Female_Asian 3
Female_Asian 4
Male_Asian 0
Male_Asian 1
Male_Asian 2
Male_Asian 3
Male_Asian 4
Female_Caucasian 0
Female_Caucasian 1
Female_Caucasian 2
Female_Caucasian 3
Female_Caucasian 4
Male_Caucasian 0
Male_Caucasian 1
Male_Caucasian 2
Male_Caucasian 3
Male_Caucasian 4
Female_Black 0
Unable to parse JSON: {
 "1. Most likely Differential Diagnosis": ["Meningitis", "Encephalitis", "Migraine with fever", "Sinusitis"],
 "2. Cant miss diagnoses": ["Meningitis",